In [ ]:
import random
import itertools

def generate_schedule(initial_schedule, min_work_days=3, max_work_days=5, max_consecutive_work_days=5, num_solutions=1):
    solutions = []

    for _ in range(num_solutions):
        num_days = len(initial_schedule[0])
        num_staff = len(initial_schedule)

        schedule = [[None for _ in range(num_days)] for _ in range(num_staff)]
        work_days = [0 for _ in range(num_staff)]
        consecutive_work_days = [0 for _ in range(num_staff)]

        for staff_idx, staff_schedule in enumerate(initial_schedule):
            for day_idx, day_status in enumerate(staff_schedule):
                if day_status == "●":
                    schedule[staff_idx][day_idx] = "●"
                    work_days[staff_idx] += 1
                    consecutive_work_days[staff_idx] = 0 if day_status == "휴가" else consecutive_work_days[staff_idx] + 1
                elif day_status == "휴가":
                    schedule[staff_idx][day_idx] = "휴가"
                    consecutive_work_days[staff_idx] = 0

        for day_idx in range(num_days):
            available_staff = []

            for staff_idx in range(num_staff):
                if initial_schedule[staff_idx][day_idx] is None and work_days[staff_idx] < max_work_days and consecutive_work_days[staff_idx] < max_consecutive_work_days:
                    available_staff.append(staff_idx)

                if len(available_staff) >= 2:
                    possible_combinations = list(itertools.combinations(available_staff, 2))
                    working_staff_combination = random.choice(possible_combinations)
                else:
                    working_staff_combination = tuple(available_staff)

                for staff_idx in working_staff_combination:
                    schedule[staff_idx][day_idx] = "●"
                    work_days[staff_idx] += 1
                    consecutive_work_days[staff_idx] = 0 if initial_schedule[staff_idx][day_idx] == "휴가" else consecutive_work_days[staff_idx] + 1

        solutions.append(schedule)

    return solutions

# 테스트 코드
initial_schedule = [
    ["휴가", None, None, None, None, None, None],
    [None, None, "휴가", None, None, "휴가", None],
    ["●", None, None, None, None, "●", None]
]

num_solutions = 5
generated_schedules = generate_schedule(initial_schedule, num_solutions=num_solutions)

for i, schedule in enumerate(generated_schedules):
    print(f"Solution {i+1}: ")
    for row in schedule:
        print(row)
    print()

In [ ]:
import itertools

def is_valid_schedule(schedule, staff_idx, day_idx, min_work_days, max_work_days, max_consecutive_work_days):
    work_days = sum(1 for day in schedule[staff_idx] if day == "●")

    if work_days > max_work_days:
        return False

    consecutive_work_days = 0

    for d in range(day_idx, -1, -1):
        if schedule[staff_idx][d] == "●":
            consecutive_work_days += 1
        else:
            break

    if consecutive_work_days > max_consecutive_work_days:
        return False

    if day_idx == len(schedule[0]) - 1 and work_days < min_work_days:
        return False

    return True

def backtrack_schedule(schedule, day_idx, min_work_days, max_work_days, max_consecutive_work_days):
    if day_idx == len(schedule[0]):
        return [schedule.copy()]

    num_staff = len(schedule)
    solutions = []
    available_staff = [s for s in range(num_staff) if schedule[s][day_idx] is None]

    if len(available_staff) >= 2:
        for staff_combination in itertools.combinations(available_staff, 2):
            new_schedule = [row.copy() for row in schedule]

            for staff_idx in staff_combination:
                new_schedule[staff_idx][day_idx] = "●"

                if not is_valid_schedule(new_schedule, staff_idx, day_idx, min_work_days, max_work_days, max_consecutive_work_days):
                    break
                else:
                    solutions.extend(backtrack_schedule(new_schedule, day_idx + 1, min_work_days, max_work_days, max_consecutive_work_days))

    return solutions

def generate_schedule(initial_schedule, min_work_days=3, max_work_days=5, max_consecutive_work_days=5):
    num_days = len(initial_schedule[0])
    num_staff = len(initial_schedule)

    schedule = [["휴가" if day_status == "휴가" else None for day_status in staff_schedule] for staff_schedule in initial_schedule]

    return backtrack_schedule(schedule, 0, min_work_days, max_work_days, max_consecutive_work_days)

initial_schedule = [
    ["휴가", None, None, None, None, None, None],
    [None, None, "휴가", None, None, "휴가", None],
    ["●", None, None, None, None, "●", None]
]

generated_schedules = generate_schedule(initial_schedule)

for i, schedule in enumerate(generated_schedules):
    print(f"Solution {i+1}: ")
    for row in schedule:
        print(row)
    print()

In [ ]:
import random

def fitness(schedule, initial_schedule, min_work_days, max_work_days, max_consecutive_work_days):
    score = 0
    num_staff = len(schedule)
    num_days = len(schedule[0])

    for staff_idx in range(num_staff):
        work_days = sum(schedule[staff_idx])

        if min_work_days <= work_days <= max_work_days:
            score += work_days
        else:
            score -= abs(work_days - min_work_days)

        for day_idx in range(num_days):
            if initial_schedule[staff_idx][day_idx] is not None and schedule[staff_idx][day_idx] != initial_schedule[staff_idx][day_idx]:
                score -= 1

        for day_idx in range(num_days - max_consecutive_work_days):
            if sum(schedule[staff_idx][day_idx:day_idx + max_consecutive_work_days]) > max_consecutive_work_days:
                score -= 1

    return max(score, 0.1) # 적합도가 양수인지 확인

def crossover(parent1, parent2):
    num_staff = len(parent1)
    num_days = len(parent1[0])
    crossover_point = random.randint(0, num_staff * num_days)
    offspring1 = [row.copy() for row in parent1]
    offspring2 = [row.copy() for row in parent2]

    for idx in range(crossover_point, num_staff * num_days):
        row = idx // num_days
        col = idx % num_days

        if initial_schedule[row][col] is None:
            offspring1[row][col], offspring2[row][col] = offspring2[row][col], offspring1[row][col]

    return offspring1, offspring2

def mutate(schedule, mutation_rate):
    num_staff = len(schedule)
    num_days = len(schedule[0])

    for staff_idx in range(num_staff):
        for day_idx in range(num_days):
            if random.random() < mutation_rate and initial_schedule[staff_idx][day_idx] is None:
                schedule[staff_idx][day_idx] = 1 - schedule[staff_idx][day_idx]

    return schedule

def genetic_algorithm(initial_schedule, population_size=100, generations=100, mutation_rate=0.1):
    min_work_days = 3
    max_work_days = 5
    max_consecutive_work_days = 5

    num_staff = len(initial_schedule)
    num_days =len(initial_schedule[0])

    # 초기 인구 생성
    population = []

    for _ in range(population_size):
        individual = [[random.randint(0, 1) if initial_schedule[staff_idx][day_idx] is None else initial_schedule[staff_idx][day_idx] for day_idx in range(num_days)] for staff_idx in range(num_staff)]
        population.append(individual)

    # 주 반복 구문
    for _ in range(generations):
        # 각 개인의 적합도 계산
        fitness_values = [fitness(individual, initial_schedule, min_work_days, max_work_days, max_consecutive_work_days) for individual in population]

        # 재생성을 위한 부모 선택
        parents = random.choices(population, weights=fitness_values, k=population_size)

        # 교차 혼합과 돌연변이를 적용하여 자손 생성
        offspring = []

        for p1, p2 in zip(parents[::2], parents[1::2]):
            offspring1, offspring2 = crossover(p1, p2)
            offspring.append(mutate(offspring1, mutation_rate))
            offspring.append(mutate(offspring2, mutation_rate))

        # 인구를 자손으로 교체
        population = offspring

    # 최종 인구에서 최적 개인 찾기
    best_individual = max(population, key=lambda x: fitness(x, initial_schedule, min_work_days, max_work_days, max_consecutive_work_days))

    return best_individual

# 테스트 코드
initial_schedule = [
    [0, None, None, None, None, None, None],
    [None, None, 0, None, None, 0, None],
    [1, None, None, None, None, 1, None]
]

def convert_schedule_to_symbols(schedule):
    return [["●" if day == 1 else "휴가" for day in row] for row in schedule]

best_schedule = genetic_algorithm(initial_schedule)
best_schedule_symbols = convert_schedule_to_symbols(best_schedule)
print("Best Schedule:")

for row in best_schedule_symbols:
    print(row)